<a href="https://colab.research.google.com/github/sjcxie/CulturallyAdapted-MentalHealthAI/blob/master/informed_multi_agent_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q crewai
!pip install -q langchain-openai
!pip install --q crewai-tools

In [2]:
import os

from google.colab import userdata
import openai
from langchain_openai import ChatOpenAI
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import PDFSearchTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

In [ ]:
# !pip install --upgrade chromadb

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('PHI_Tier2_ChineseChatbot_Project')
llm = ChatOpenAI(model="gpt-4o")

In [48]:
pdf_path = 'knowledge/华人活动.pdf'
pdf_search_tool = PDFSearchTool(pdf=pdf_path)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [49]:
response_critic_knowledge_base = Agent(
  role='基于知识库的评估专家',
  goal="""根据客户的输入内容，"{client_message}" ，评估心理顾问的回复："{therapist_response}"。使用来自 {pdf_paths} 的知识库，识别缺失的文化元素。""",
  backstory="""凭借对{cultural_background}的深刻理解和对全面知识库的访问，您专注于评估心理顾问的回复，确保其反应了{cultural_background}背景。""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[pdf_search_tool]
)
response_adapter = Agent(
  role='文化背景融合专家',
  goal='修改心理顾问的回复, "{therapist_response}"，使其融入缺失的文化元素。',
  backstory="""作为文化背景融合专家，您擅长根据特定的文化背景定制沟通内容，确保心理顾问的回复既既保留了心理顾问的专业性又有效融入了文化元素。""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)
critique_therapist_response_knowledge_base_task = Task(
  description="""使用 PDFSearchTool 访问 {pdf_paths}。结合客户的输入内容, "{client_message}"，评估当前心理顾问的回复 "{therapist_response}"。
  识别和客户的输入或和心理顾问的回复相关的文化元素并提供改进建议。""",
  expected_output="""一份中文评估报告，指出当前心理顾问的回复， "{therapist_response}"， 中缺失的文化元素，并附上改进建议。""",
  async_execution=False,
  agent=response_critic_knowledge_base
)
adapt_therapist_response = Task(
  description="""根据评估报告以及客户的输入内容 "{client_message}" ，修改心理顾问的回复 "{therapist_response}"。
  定制心理顾问的回复，确保其保留了原心理顾问的意图并且符合客户的文化背景。仅输出调整后的回应。""",
  expected_output="""调整后的心理顾问回复，应保留原心理顾问回复中的意图并且更加符合客户的文化背景。""",
  async_execution=False,
  agent=response_adapter,
  context=[critique_therapist_response_knowledge_base_task]
)

In [50]:

culture = "中国文化"
# client_message = "'I am stressed and constantly sense of responsibility to respect and care for aging parents'"
client_message = "家人对我有很多要求和期望，但从来没有人关心过我，问过我想要什么样的生活。"
# therapist_response = "'I understand how you feel, it could be the sense of obligation that is causing you the stress.'"
therapist_response = "听起来你正在承受着很大的压力，而没有得到应有的关注和支持。你有尝试过一些放松的活动，比如听音乐或者做瑜伽吗？"


In [51]:
inputs={'cultural_background': culture,
        'client_message': client_message,
        'therapist_response': therapist_response,
        'pdf_paths': pdf_path}

In [52]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[response_critic_knowledge_base, response_adapter],
  tasks=[critique_therapist_response_knowledge_base_task, adapt_therapist_response],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [53]:
crew

Crew(id=85377c2b-165e-484f-9a82-f9138b820746, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

In [54]:
result = crew.kickoff(inputs=inputs)

# Agent: 基于知识库的评估专家
## Task: 使用 PDFSearchTool 访问 knowledge/华人活动.pdf。结合客户的输入内容, "家人对我有很多要求和期望，但从来没有人关心过我，问过我想要什么样的生活。"，评估当前心理顾问的回复 "听起来你正在承受着很大的压力，而没有得到应有的关注和支持。你有尝试过一些放松的活动，比如听音乐或者做瑜伽吗？"。
  识别和客户的输入或和心理顾问的回复相关的文化元素并提供改进建议。


# Agent: 基于知识库的评估专家
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"family expectations support relaxation activities Chinese culture\"}"
## Tool Output: 
Relevant Content:
社 交 活 动 一、 传 统 节 庆 与 家 庭 聚 会 1. 节 日 聚 餐 ○ 适 用 群 体 ： 全 体 华 人 家 庭 （尤 其 注 重 传 统 的 中 老 年 人 与 多 代 同 堂 家 庭） ○ 理 由 ： 通 过 共 享 节 日 仪 式 感 强 化 家 族 认 同 ， 满 足 “ 团 圆 ” 情 感 需 求 ， 符 合 儒 家 “ 家 和 万 事 兴 ” 价 值 观。 2. 祭 祀 与 宗 族 活 动 ○ 适 用 群 体 ： 宗 族 观 念 强 的 群 体 （如 南 方 宗 亲 会 成 员、 海 外 华 人 寻 根 群 体） ○ 理 由 ： 通 过 祭 祖 维 系 血 缘 纽 带 ， 增 强 家 族 归 属 感 ， 尤 其 在 移 民 群 体 中 可 缓 解 文 化 断 层 焦 虑。 二、 饮 食 文 化 为 核 心 的 社 交 1. 茶 楼 饮 早 茶 / 下 午 茶 ○ 适 用 群 体 ： 中 老 年 人、 商 务 人 士、 家 庭 主 妇 ○ 理 由 ： 以 茶 点 为 媒 介 的 “ 慢 社 交 ” 符 合 华 人 “ 饭 局 谈 事 ” 习 惯 ， 非 正 式 场 合 降 低 沟 通 压 力。 2. 夜 市 与 大 排 档 聚 餐 ○ 适 用 群 体 ： 年 轻 人、 工 薪 阶 层、 夜 生 活 爱 好 者 ○ 理 由